In [ ]:
import csv
import os
import re
import pandas as pd
import json
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from dotenv import load_dotenv

In [ ]:
# load credentials from .env file
load_dotenv()

CLIENT_ID = os.getenv("CLIENT_ID", "")
CLIENT_SECRET = os.getenv("CLIENT_SECRET", "")
OUTPUT_FILE_NAME = "lukewarm_playlist.csv"

# change for your target playlist
playlist_link = "https://open.spotify.com/playlist/3W0kChA5K4tozflcfjFmzv?si=4b50754f3f914f9e"

In [ ]:
# authenticate
client_credentials_manager = SpotifyClientCredentials(
    client_id=CLIENT_ID, client_secret=CLIENT_SECRET
)

# create spotify session object
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [ ]:
# get playlist uri from https link
playlist_uri = playlist_link.split("/")[-1].split("?")[0]

# get list of tracks in a given playlist (note: max playlist length 100)
#tracks = session.playlist_tracks(playlist_uri)["items"]

# get list of track URIs
#track_uris = [x["track"]["uri"] for x in session.playlist_tracks(playlist_uri)["items"]]

In [ ]:
def get_playlist_tracks(playlist_uri):
    results =  sp.playlist_tracks(playlist_uri)
    tracks = results['items']
    while results['next']:
        results = sp.next(results)
        tracks.extend(results['items'])
    return tracks


In [ ]:
df = pd.DataFrame(columns=["track_uri","track_name","track_pop",
                           "artist_uri","artist_info", "artist_name", 
                           "artist_pop", "artist_genres", "album_name"]) 


In [ ]:
for track in get_playlist_tracks(playlist_uri): 
    track_uri = track['track']['uri']
    track_name = track['track']['name']
    artist_uri = track['track']['artists'][0]['uri']
    artist_info = sp.artist(artist_uri)
    artist_name = track['track']['artists'][0]['name']
    artist_pop = artist_info['popularity']
    artist_genres = artist_info['genres']
    album_name = track['track']['album']['name']
    track_pop = track['track']['popularity']
    
    df = df.append({'track_uri':track_uri,'track_name':track_name, 'track_pop':track_pop,
                'artist_uri':artist_uri,'artist_info':artist_info, 'artist_name':artist_name, 
                'artist_pop':artist_pop, 'artist_genres':artist_genres, 'album_name':album_name},ignore_index=True)
 

In [ ]:
df.head()

In [ ]:
len(df)

In [ ]:
#df = df.drop([0])
#df = df.reset_index()
#df.to_csv('playlist_raw.csv')